# Notebook File Overview

In this notebook, we will be importing the data frames we created in the first two notebooks:
1. diabetes_control_ptsd.csv
2. diabetes_study_ptsd.csv

We will use the code snippet provided by *All of Us* to import our data frames from the workspace bucket (looks similar to the snippet used to save our data frames to the workspace bucket). 

We will do some data processing and cleaning on our existing data frames including:
* Create a new variable to designate which participants **have** and **do not have** diabetes 
* Combine the two dataframes into one dataframe
* Calculate participant age based on the current date and their DOB and create a new variable called *age*
* Select the final columns we want for our final dataframe we will analyze

# Add the code snippet from the *All of Us R and Cloud Storage snippets*

**Step 1: Run *Setup***

In [ ]:
library(tidyverse)  # Data wrangling packages.

**Step 2: Run the *copy_file_from_workspace_bucket.R* code snippet**

This will import **diabetes_control_ptsd.csv** and **diabetes_study_ptsd.csv** as data frames

**NOTE: You will have to add another line of code to each step to get both files imported**

In [ ]:
# This snippet assumes that you run setup first

# This code copies a file from your Google Bucket into a dataframe

# replace 'test.csv' with the name of the file in your google bucket (don't delete the quotation marks)
diabetes_control_ptsd <- 'diabetes_control_ptsd.csv'
diabetes_study_ptsd <- 'diabetes_study_ptsd.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
#######################################################################

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ", my_bucket, "/data/", diabetes_control_ptsd, " ."), intern=T)
system(paste0("gsutil cp ", my_bucket, "/data/", diabetes_study_ptsd, " ."), intern=T)

# Load the file into a dataframe
diabetes_control_ptsd  <- read_csv(diabetes_control_ptsd)
diabetes_study_ptsd  <- read_csv(diabetes_study_ptsd)

head(diabetes_control_ptsd)
head(diabetes_study_ptsd)

# Clean and join the two dataframes

**Step 1: Add new columns to each data frame called *diabetes* and include values based on diabetes status**

We will use the mutate() function from the **dplyr** package to create new variables for each data frame.

In [ ]:
library(dplyr)

diabetes_no_ptsd <- diabetes_control_ptsd %>%
  mutate(diabetes = "No")

diabetes_yes_ptsd <- diabetes_study_ptsd%>%
  mutate(diabetes = "Yes")

dim(diabetes_no_ptsd)
head(diabetes_no_ptsd)
dim(diabetes_yes_ptsd)
head(diabetes_yes_ptsd)

**Step 2: Combine data frames and finalize dataset**

We will use several functions from the **dplyr** package to create our final analysis-ready dataset:

* Use the *bind_rows()* function to combine the diabetes_no_ptsd and diabetes_yes_ptsd data frames into one dataset
* Use the *rename()* function to give shorter, more manageable names to the long PTSD survey question columns
* Use the *mutate()* function to calculate participant age from their date of birth and create a new variable called *age*
* Use the *select()* function to reorder columns and remove the original date_of_birth column since we now have the calculated age


We use bind_rows() instead of a join function because the column names for each data frame are the same and in the same order, so we can simply stack the data together which is easier, cleaner, and has fewer potential errors

In [ ]:
final_diabetes_ptsd <- bind_rows(diabetes_no_ptsd, diabetes_yes_ptsd) %>%
  rename(
    ptsd_doctor = `Are you still seeing a doctor or health care provider for post-traumatic stress disorder (PTSD)?`,
    ptsd_treatment = `Are you currently prescribed medications and/or receiving treatment for post-traumatic stress disorder (PTSD)?`
  ) %>%
  mutate(
    age = floor(interval(ymd_hms(date_of_birth), today()) / years(1))
  ) %>%
select(person_id, age, everything(), -date_of_birth)



dim(final_diabetes_ptsd)
head(final_diabetes_ptsd)

# Add the code snippet from the *All of Us R and Cloud Storage snippets*

**Run the *copy_file_to_workspace_bucket.R* code snippet**

In [ ]:
# This snippet assumes that you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe <- final_diabetes_ptsd

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename <- 'final_diabetes_ptsd.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# store the dataframe in current workspace
write_excel_csv(my_dataframe, destination_filename)

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)

# Check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/*.csv"), intern=T)
